In [1]:
!pip install openai==0.27.8
!pip install --upgrade PyPDF2
!pip install tiktoken
!pip install langchain
!pip install sentence_transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# imports
import openai
import langchain
from langchain.text_splitter import TextSplitter, CharacterTextSplitter
import PyPDF2
import requests

In [3]:
openai.api_key = ""
openai_params = {"model":"gpt-4-1106-preview",
                 "temperature":0.5,
                 "frequency_penalty":0.0,
                 "presence_penalty":0.0,
                 "max_tokens":1500,
                 "top_p":1}

openai_params1 = {"model":"gpt-3.5-turbo-1106",
                 "temperature":0.5,
                 "frequency_penalty":0.0,
                 "presence_penalty":0.0,
                 "max_tokens":1500,
                 "top_p":1}

In [4]:
def generate_answer(prompt,openai_params):
  message = [{"role":"user","content":prompt}]
  response = openai.ChatCompletion.create(messages=message,
                                        **openai_params)

  return response.choices[0].message.content

In [5]:
def generate_summary(text):
  prompt = file_description_prompt(text)
  message = [{"role":"user","content":prompt}]
  response = openai.ChatCompletion.create(messages=message,
                                        **openai_params1)

  return response.choices[0].message.content


In [6]:
import tiktoken
def count_tokens(input):

    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(input))

def truncate_to_token_limit(input,limit):
    enc = tiktoken.get_encoding("cl100k_base")
    return enc.decode(enc.encode(input)[:limit])

In [7]:
# file_description
def file_description_prompt(text):
  prompt = f"""Your task is to craft a summary of the provided text, aiming for a length of 140-150 words. This compact summary should distill the text to its core elements and serve as a basis for formulating efficient search queries. Your summary should concisely include:

Main themes and subjects, encapsulated in a brief overview.
Key details, examples, or notable elements, described succinctly.
Relevant additional context or related topics, briefly touched upon.
Ensure that the summary is comprehensive yet concise, ideally within the 140-150 word range, to effectively capture the text's essence for subsequent search query development.

Text for Summarization:
{text}

140-150 Word Concise Summary:
"""
  return prompt

In [8]:

def query2doc_prompt(query):
  _prompt = f"""Write a passage that answers the given query:

    Query: what state is this zip code 85282
    Passage: Welcome to TEMPE, AZ 85282. 85282 is a rural zip code in Tempe, Arizona. The population
    is primarily white, and mostly single. At $200,200 the average home value here is a bit higher than
    average for the Phoenix-Mesa-Scottsdale metro area, so this probably isn’t the place to look for housing
    bargains.5282 Zip code is located in the Mountain time zone at 33 degrees latitude (Fun Fact: this is the
    same latitude as Damascus, Syria!) and -112 degrees longitude.

    Query: why is gibbs model of reflection good
    Passage: In this reflection, I am going to use Gibbs (1988) Reflective Cycle. This model is a recognised
    framework for my reflection. Gibbs (1988) consists of six stages to complete one cycle which is able
    to improve my nursing practice continuously and learning from the experience for better practice in the
    future.n conclusion of my reflective assignment, I mention the model that I chose, Gibbs (1988) Reflective
    Cycle as my framework of my reflective. I state the reasons why I am choosing the model as well as some
    discussion on the important of doing reflection in nursing practice.

    Query: what does a thousand pardons means
    Passage: Oh, that’s all right, that’s all right, give us a rest; never mind about the direction, hang the
    direction - I beg pardon, I beg a thousand pardons, I am not well to-day; pay no attention when I soliloquize,
    it is an old habit, an old, bad habit, and hard to get rid of when one’s digestion is all disordered with eating
    food that was raised forever and ever before he was born; good land! a man can’t keep his functions
    regular on spring chickens thirteen hundred years old.

    Query: what is a macro warning
    Passage: Macro virus warning appears when no macros exist in the file in Word. When you open
    a Microsoft Word 2002 document or template, you may receive the following macro virus warning,
    even though the document or template does not contain macros: C:\<path>\<file name>contains macros.
    Macros may contain viruses.

    Query: {query}
    Passage:
 """
  return _prompt.strip()

In [10]:

def tokenize_text_gpt(content,chunk_size=120,splitter_pattern=""):
    """
    Tokenize the text according to openai tokenizer using Langchain
    :param content:
    :return:
    """
    if not splitter_pattern:

        if "\n\n" in content:

            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,encoding_name="cl100k_base")
        elif "\n" in content:
            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,
                                                                         separator="\n",encoding_name="cl100k_base")
        else:
            text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=0,
                                                                         separator=" ",encoding_name="cl100k_base")
    else:
        text_splitter_ = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size,chunk_overlap=0,
                                                                    separator=splitter_pattern,encoding_name="cl100k_base")
    passages = text_splitter_.split_text(content)

    return passages

In [11]:
def extract_text_from_pdf(pdf_file_path):
    # Open the PDF file
    with open(pdf_file_path, 'rb') as file:
        # Create PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        extracted_text = ""

        for page in pdf_reader.pages:

            extracted_text += page.extract_text()

        return extracted_text


In [12]:
pdf_path = ["/content/1507.05717.pdf","/content/2104.06022.pdf","/content/2104.06022.pdf","/content/2209.03592.pdf","/content/2303.10420.pdf","/content/2305.09972.pdf","/content/paper 1.pdf","/content/105-Workflow-Automation-Use-Case-Ideas-1.pdf","/content/2023-gartner-cio-agenda-ebook.pdf","/content/2023-gartner-top-strategic-technology-trends-ebook.pdf"]
pdf_chunks = {}
summary_pdfs = {}
for i in pdf_path:
  text = extract_text_from_pdf(i)
  print(len(text))
  if count_tokens(text) > 16000:
    summary_txt = generate_summary(truncate_to_token_limit(text,14000))
  else:
    summary_txt = generate_summary(text)
  summary_pdfs[i.split('/')[-1]] = summary_txt
  print(summary_txt)
  chunks = tokenize_text_gpt(text,chunk_size=256)
  print(len(chunks))
  chunks_modf = []
  for j in chunks:
    j = "<CHUNK>" +" : " +  j + "</CHUNK>" +"\n\n" +"<PDF SUMMARY>"+" : " + summary_txt + "</PDF SUMMARY>"
    chunks_modf.append(j)

  pdf_chunks[i.split('/')[-1]] = chunks_modf

43273
The paper introduces a novel neural network architecture called Convolutional Recurrent Neural Network (CRNN) for image-based sequence recognition. CRNN integrates feature extraction, sequence modeling, and transcription into a unified framework, making it end-to-end trainable and capable of handling sequences of arbitrary lengths without predefined lexicons. The proposed algorithm outperforms prior arts in scene text recognition and also demonstrates promising results in image-based music score recognition. CRNN requires minimal domain knowledge and is suitable for various image-based sequence recognition tasks. The paper provides a comprehensive comparison with other text recognition approaches, highlighting the advantages of CRNN in terms of end-to-end trainability, use of convolutional features, and model size. Additionally, the paper discusses the impact of parameter settings and presents results from musical score recognition, where CRNN outperforms commercial Optical Music

In [13]:
pdf_chunks['105-Workflow-Automation-Use-Case-Ideas-1.pdf'][0]

'<CHUNK> : Workflo w\nAutomation Use-Case Ideas\nTO  YOUR  10x\nPRODUCTIVITY\nAT WORK1054 Accounts and Finance\n10 Administration & Operations\n14 Customer Service\n17 Field Services\n21 Human Resources \n26 IT Operations \n31 Legal\n35 Procurement\n39 Project Management \n43 Research and Development\n47 Sales and Marketing\n52 Product Management\nSecurity\nFacilities Management56\n60Index\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n2Introduction 3Introduction\nThe traditional app development process starts with \nemployees explaining their requirements to the software \ndevelopers. And then, according to their understanding of the \nproblem, software developers build solutions that could help \nbusiness users. But what is lacking in this process is that the one \nwho has \uf001rst-hand experience with the problem (business users) \nhas no participation in the actual building of the solution.\nNo-code work\uf002ow automation has emerged as a diamond in \nthe rough for business user

In [14]:
for i in summary_pdfs.keys():
  print((summary_pdfs[i]))

The paper introduces a novel neural network architecture called Convolutional Recurrent Neural Network (CRNN) for image-based sequence recognition. CRNN integrates feature extraction, sequence modeling, and transcription into a unified framework, making it end-to-end trainable and capable of handling sequences of arbitrary lengths without predefined lexicons. The proposed algorithm outperforms prior arts in scene text recognition and also demonstrates promising results in image-based music score recognition. CRNN requires minimal domain knowledge and is suitable for various image-based sequence recognition tasks. The paper provides a comprehensive comparison with other text recognition approaches, highlighting the advantages of CRNN in terms of end-to-end trainability, use of convolutional features, and model size. Additionally, the paper discusses the impact of parameter settings and presents results from musical score recognition, where CRNN outperforms commercial Optical Music Recog

In [15]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# 1. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/111k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [78]:
sentence_embeddings = model.encode(pdf_chunks['105-Workflow-Automation-Use-Case-Ideas-1.pdf'])
sentence_embeddings.shape[0]

(39, 1024)

In [84]:
np.array(sentence_embeddings).shape[0]

39

In [24]:
# from sentence_transformers import SentenceTransformer
# # initialize sentence transformer model
# model = SentenceTransformer('bert-base-nli-mean-tokens')
# # create sentence embeddings
# sentence_embeddings = model.encode(fin_chunks)
# sentence_embeddings.shape

(631, 768)

In [30]:
def get_embedding_list(texts, model="text-embedding-ada-002"):
  texts = [re.sub("\n+", " ", text) for text in texts]
  embedding_data = openai.Embedding.create(input = texts, model=model)['data']
  print("embeddings returned from openai")
  return [embedding_data[i]["embedding"] for i in range(len(embedding_data))]



def get_embedding(text, model="text-embedding-ada-002"):
  text = re.sub("\n+", " ", text)
  return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [16]:
import numpy as np
import re
def get_embedding_list(texts):
    texts = [re.sub("\n+", " ", text) for text in texts]
    # model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
    try:
        print("calling sentence transformer")
        embeddings = model.encode(texts)

        return embeddings
    except Exception as e:
        print(e)
        return []

def get_embedding(text):
    text = re.sub("\n+", " ", text)
    model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
    try:
        print("calling sentence transformer")
        embeddings = model.encode(text)
        return embeddings
    except Exception as e:
        print(e)
        return []

In [35]:
def create_index(text_chunks):

    all_chunks = []
    id_to_pdf_and_chunk = {}
    current_id = 0

    for pdf_name, chunks in pdf_chunks.items():
        for i, chunk in enumerate(chunks):
            all_chunks.append(chunk)
            id_to_pdf_and_chunk[current_id] = (pdf_name, i)
            current_id += 1

    embeddings = get_embedding_list(all_chunks)
    if len(embeddings) == 0:
        print("No embeddings generated.")
        return None, {}

    dimension = len(embeddings[0])  # Dynamically get the dimension of embeddings

    index = hnswlib.Index(space='l2', dim=dimension)
    index.init_index(max_elements=len(all_chunks), ef_construction=200, M=16)

    # Bulk adding to the index
    index.add_items(embeddings)

    index.set_ef(50)
    return index, id_to_pdf_and_chunk

def create_index2(text_chunks):

    all_chunks = []
    id_to_pdf_and_chunk = {}
    current_id = 0

    for pdf_name, chunks in pdf_chunks.items():
        for i, chunk in enumerate(chunks):
            all_chunks.append(chunk)
            id_to_pdf_and_chunk[current_id] = (pdf_name, i)
            current_id += 1

    embeddings = get_embedding_list(all_chunks)
    if len(embeddings) == 0:
        print("No embeddings generated.")
        return None, {}

    dimension = len(embeddings[0])  # Dynamically get the dimension of embeddings

    index = hnswlib.Index(space='l2', dim=dimension)
    index.init_index(max_elements=len(all_chunks), ef_construction=200, M=16)

    # Bulk adding to the index
    index.add_items(embeddings)

    index.set_ef(50)
    return index, id_to_pdf_and_chunk

def search_similar_text(query, index, id_to_pdf_and_chunk, top_k):
    query_vector = get_embedding(query)
    try:
        labels, distances = index.knn_query(query_vector, k=top_k)

        # Flatten the labels and distances since we have a single query
        labels = labels.flatten()
        distances = distances.flatten()

        # Map labels back to PDF names and chunk indices
        results = [(id_to_pdf_and_chunk[label], distance) for label, distance in zip(labels, distances)]

    except Exception as e:
        print(e)
        results = []
    return results


In [36]:
index,id_to_pdf = create_index(pdf_chunks)

embeddings returned from openai


In [37]:
query = "What is ordering of text in extraction of multimodal context"
query_modf = generate_answer(query2doc_prompt(query),openai_params)
query_modf = "<QUERY>"+ " : " +  query + "</QUERY>" + "\n\n" + "<HYPOTHETICAL ANSWER>"+ " : " + query_modf + "</HYPOTHETICAL ANSWER>"
results = search_similar_text(query,index,id_to_pdf,60)

In [38]:
query_modf

'<QUERY> : What is ordering of text in extraction of multimodal context</QUERY>\n\n<HYPOTHETICAL ANSWER> : The ordering of text in the extraction of multimodal context refers to the process of arranging and interpreting text in conjunction with other modes of information, such as images, audio, and video, to understand the overall meaning within a multimodal dataset or document. This process is crucial in fields such as natural language processing (NLP), where algorithms are designed to detect and understand the relationships between different types of content to provide accurate information retrieval, sentiment analysis, or content summarization.\n\nWhen extracting information from multimodal sources, the order in which text appears relative to other elements can significantly impact the interpretation of the data. For instance, captions under images, subtitles on videos, or text descriptions of audio content must be aligned with the corresponding media to maintain the correct context

In [39]:
# pdf_names = []
# for result in results:
#     pdf_name, chunk_index = result[0]
#     pdf_names.append(pdf_name)
#     distance = result[1]
#     print(f"PDF: {pdf_name}, Chunk Index: {chunk_index}, Distance: {distance}")

pdf_counts = {}
for result in results:
    pdf_name, chunk_index = result[0]
    distance = result[1]
    # Count occurrences of each PDF name
    if pdf_name not in pdf_counts:
        pdf_counts[pdf_name] = 1
    else:
        pdf_counts[pdf_name] += 1
k = 20
pdf_names = [pdf_name for pdf_name, count in pdf_counts.items() if count > k]


In [ ]:
results

In [40]:
pdf_counts

{'paper 1.pdf': 60}

In [42]:
fin_pdfs = set(pdf_names)

In [43]:

def query_prompt_multi_file(Statement,filenames_with_descriptions):

    _prompt = f"""In this task, you are responsible for creating search queries that enable the efficient extraction of relevant information from specified files. Use the filenames and their short descriptions, as well as the instruction provided, to tailor your queries.
#### File List
{filenames_with_descriptions}
#### Instruction
{Statement}
#### Objectives
Your queries should aim to:
1. Identify the core theme or subject of the instruction within the file.
2. Pinpoint specific details, examples, or aspects that directly address the instruction.
3. Uncover any additional context or related information that could enrich the response to the instruction.

#### Note
Since you are aware of the filenames, there's no need to include them or the Statement in your queries or to query any external search engine.
Generate upto 10 queries. The queries should cover all the topics.

#### Your Queries
"""
    return _prompt

In [44]:
filenames_with_descriptions = {}
documents = []
for i in fin_pdfs:
  print(i)
  print(summary_pdfs[i])
  documents.extend(pdf_chunks[i])
  filenames_with_descriptions[i] = summary_pdfs[i]

paper 1.pdf
This paper introduces a multimodal context-aware translation framework and automatic corpus construction for manga translation. The main challenges in manga translation are the unstructured layout of text and images, as well as the lack of context-aware and multimodal translation models. The authors propose a method to automatically construct a large parallel corpus from original manga and their translations, and create a new benchmark dataset for manga translation. They also develop a fully automated manga translation system that recognizes text, translates it into the target language, and replaces the original text with the translated version. The system is evaluated on various datasets, showing significant improvement over existing methods. Future work includes improving image recognition models and extending the scene-based NMT to capture longer contexts.


In [54]:
len(documents)

69

In [ ]:
documents

In [50]:
index1,id_to_pdf1 = create_index(documents)

embeddings returned from openai


In [ ]:
id_to_pdf1

In [51]:
prompt = query_prompt_multi_file(Statement = query,filenames_with_descriptions = filenames_with_descriptions)
fin_chunks_qa_ans = []
for i in generate_answer(prompt,openai_params1).split('\n'):
  results = search_similar_text(i,index1,id_to_pdf1,20)
  fin_chunks_qa_ans.append(results)

In [ ]:
fin_chunks_qa_ans

In [ ]:
filenames_with_descriptions

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder("mixedbread-ai/mxbai-rerank-large-v1")

In [48]:
# reranking
%time
prompt = query_prompt_multi_file(Statement = query,filenames_with_descriptions = filenames_with_descriptions)
fin_chunks_qa_ans = []
for i in generate_answer(prompt,openai_params1).split('\n'):
  results = model.rank(i, documents, return_documents=True, top_k=3)
  fin_chunks_qa_ans.append(results)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


KeyboardInterrupt: 

In [ ]:
fin_chunks_qa_ans

In [17]:
!pip install hnswlib
import hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287616 sha256=a92ced4cb2827dde7193373bd3ae72448d8bbf27f053ca1437abb98903362f36
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib
